# Presentación

## Tecnicatura Universitaria en Inteligencia Artificial

### Procesamiento de lenguaje natural - IA.42

---

**Estudiante**: Fabio Giampaoli


**Fecha**: 18-12-2023

---

Este proyecto consiste en la creación de un modelo de lenguaje como asistente para conversar con lenguaje natural sobre algún dominio de interés a alección, y la investigación de la posibilidad de convertirlo en un sistema multiagente que pueda conectar con diversas herramientas.

En este caso, mi interés es que el agente sea experto en Genexus. Genexus en un entorno de desarrollo de software que mediante un lenguaje de programación simiplificado e integraciones con otras herramientas, se puede generar código fuente para compilar y desplegar aplicaciones.

El objetivo es que el agente pueda tener a disposición la documentación oficial de Genexus para que pueda mantener una conversación al respecto como si Genexus fuera parte de su fuente de su conocimiento, para finalmente darle al usuario que interactua la sensación de que el agente puede ayudar a resolver dudas y problemas relacionados.

# Resolución

### Entorno


In [ ]:
%%capture
# instalación de modulos
!pip install -q chromadb
!pip install sentence-transformers langchain
!pip install -U deep-translator

# procesamiento
import pandas as pd
import torch
import re

# scrapping
from bs4 import BeautifulSoup
import requests

# limpieza
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer

# tokenización
import nltk
from nltk import sent_tokenize

# base de datos vectorial
import chromadb
from sentence_transformers import SentenceTransformer, util

# traducción de texto
from deep_translator import GoogleTranslator

# modelo de embedding para oraciones
modelo_embedding = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# descargas
nltk.download('stopwords')
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download('punkt')

### Fuentes

Para el objetivo de alimentar modelos de lenguaje con información de como funciona Genexus se recurre a la documentación oficial de su sitio oficial: La wiki de Genexus (https://wiki.genexus.com/commwiki/wiki?1756,Category%3AGeneXus).


La estructura de la wiki es particularmente conveniente para este escenario. Podemos notar que la estructura html de la página se puede extraer facilmente el contenido textual y el código y sintáxis de los ejemplos. De hecho, hay una divisor particular con la clase "TableWikiBody" que contiene toda esta información. Por lo que extrear el texto de este tag particular retorna todo el contenido buscado.

In [ ]:
def scrape(url):
    '''Extrae el texto del divisor con la clase que abarca el contenido de la página'''
    # consulta al link de la página
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # encuentra el divisor deseado
    wiki_body = soup.find_all("div", class_="TableWikiBody")[0]

    # extrae todo el texto del divisor
    extracted_text = []
    for element in wiki_body.find_all(recursive=False):
        extracted_text.append(element.text.strip())

    # concatena a un unico string
    texto_final = '\n'.join(extracted_text)
    return texto_final

Además, cuenta con un índice de contenidos a la izquieda que es una estructura con desplegables que te llevan a las diferentes paginas de la Wiki para leer sobre los diferentes temas de interés.

Cada elemento del indice representa el título de la página y una referencia al link de dicha página. Por lo que iterar sobre esta estructura tambien resulta muy simple.

In [ ]:
# consulta a la wiki
url = "https://wiki.genexus.com/commwiki/wiki?6426,Inline+Formulas+within+a+contextual+table"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# obtiene los sub-elementos del indice bajo el titulo Genexus
menu = soup.find_all('li')[3].find_all('a')

# para guardar en dataframe el indice
refs = []
titles = []
df_menu = pd.DataFrame()

# extraer de cada elemento el titulo y su url completa
for ref in menu:
    refs.append(f"https://wiki.genexus.com/{ref.get('href')}")
    titles.append(ref.text)

df_menu['Title'] = titles
df_menu['Link'] = refs
df_menu['Content'] = df_menu['Link'].apply(scrape) # de cada link extrae su contenido

Para contextualizar de mejor manera el contenido podemos asignar categorias a cada registro de este nuevo dataframe. De forma que la categoria abarque determinados tipos de información dentro.

In [ ]:
def classify(titles, classes, start_title, end_title, class_label):
    '''detecta el inicio y fin de los indices correspondientes a los titulos, y les asigna la categoria a ambos y sus intermedios'''
    start_index = None
    end_index = None

    # extrae el titulo de la url de cada registro
    for i, tag in enumerate(titles):
        soup = BeautifulSoup(tag.text, 'html.parser')
        title_text = soup.get_text(strip=True)

        # compara para determinar su indice
        if start_title == title_text:
            start_index = i
        elif end_title == title_text:
            end_index = i

    # asigna el valor de clase
    if start_index is not None and end_index is not None:
        for i in range(start_index, end_index + 1):
            classes[i] = class_label

# lista vacia para las clases
classes = [None for _ in range(len(menu))]

# selección manual de inicio y fin de las categorias
classify(menu, classes, 'What is a Knowledge Base', 'GeneXus IDE', 'General')
classify(menu, classes, 'Transaction object', 'File object', 'Modeling')
classify(menu, classes, 'Reorganization', 'Impact Database Tables', 'Building')
classify(menu, classes, 'Data View object', 'Payment services', 'Integration')
classify(menu, classes, 'Knowledge Manager', 'Application Security', 'Knowledge')

df_menu['Class'] = classes

<ipython-input-24-c6dccb923d7e>:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(tag.text, 'html.parser')


Por lo que contamos ahora con un dataframe que nos permite acceder a todo el contenido de la Wiki de Genexus y conocer sus repectivos titulos, links y clases.

In [ ]:
display(df_menu.head(3))
display(df_menu.shape)

,Title,Link,Content,Class,Cleaned
0,What is a Knowledge Base,"https://wiki.genexus.com//commwiki/wiki?1836,K...",To create a new software system with GeneXus y...,General,create new software system genexus must create...
1,Knowledge Base Creation,"https://wiki.genexus.com//commwiki/wiki?9596,N...",You can create a new Knowledge Base by selecti...,General,create new knowledge base select file > new > ...
2,GeneXus IDE,"https://wiki.genexus.com//commwiki/wiki?5272,C...",GeneXus provides an intuitive and consistent U...,General,genexus provide intuitive consistent user inte...


(140, 5)

Con esto se cubre el requisito mínimo de al menos 100 páginas de texto de tres documentos diferentes.

### Depuración

La etapa de limpiza del texto es sencilla debido a que el texto contiene código, y la eliminación o alteración del código dentro del texto puede tener un impacto negativo al final debido a que podrian ser interpredos palabras del lenguaje de programación que han sido procesadas como si fueran parte del lenguaje.

Además, todo el contenido proviene de la documentación oficial. Por lo que podemos asumir que no hay espacios vacios, incosistencias en las explicaciones, o errores ortigraficos.

In [ ]:
def clean_text(text):
    text = text.lower() # normalizar en minisculas
    # text = text.translate(str.maketrans('', '', string.punctuation)) # eliminar signos de puntuacion (no rinde bien)
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')]) # eliminar palabras de parada
    text = ' '.join([WordNetLemmatizer().lemmatize(word, pos="v") for word in text.split()]) # para llevar las palabras a su forma base

    return text

df_menu['Cleaned'] = df_menu['Content'].apply(clean_text)

In [ ]:
# ejemplo de un registro depurado
df_menu['Cleaned'].iloc[45]

'inline formula local formula define within object code (for example middle procedure source, object subroutine, data provider statement, etc.). words, formula locally assign attribute variable, sentence statement middle code, clause, etc. syntax <variable>|<attribute> = <unconditionalformula>} | <unconditionalformula> inline formula (<unconditionalformula>), either horizontal aggregate. main difference regard global formulas inline may consist several conditional expressions, even single conditional one. addition, inline formula, variables could involve expression (because formula exist local piece code variable known). whereas really know place global formula go triggered. mean "global". use everywhere attribute correspond base table allowed) global formulas always table context (the table formula attribute would store virtual), inline formulas to. kind formulas? need context evaluated: aggregate formulas. context meaningful? affect calculation result, allow add filter condition data

Vemos por ejemplo que no eliminar los signos de puntación hace que las lineas de código no pierdan sentido y en el futuro los modelos puedan interpretar estas palabras y simbolos como parte del lenguaje.

In [ ]:
df_menu.to_csv('genexus-wiki.csv')

### Base de Datos Vectorial

Se utiliza un base de datos vectorial para almacenar todo este conocimiento 'CromaDB' debido a la facilidad de uso y practicidad.

Se define que los documentos seran todos los registros del dataframe depurados, sus indices como strings unicos, y su metadata serán las categorias, url's y títulos.

In [ ]:
# df_menu = pd.read_csv('genexus-wiki.csv')

metadata = [{'Link': row['Link'], 'Title': row['Title'], 'Class': row['Class']} for i, row in df_menu.iterrows()]
documents = [row['Cleaned'] for i, row in df_menu.iterrows()]
ids = [f'id{i+1}' for i in df_menu.index]

In [ ]:
# creación de la base
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="genexus-wiki-tab")

# añiade toda la información
collection.add(
    documents=documents,
    metadatas=metadata,
    ids=ids
)

Podemos probar por ejemplo una pregunta sobre Genexus a esta base de datos y la misma será capaz de buscar cual es el documento que mas se parace a la pregunta utilizando métricas de distancia para medir cercanias en un espacio que captura las representaciones vectoriales de cada documento.

In [ ]:
user_query = 'what is the variable &trnContext used in trasactions events?'

# traer los dos documentos mas cercanos a la pregunta
res = collection.query(
    query_texts=[user_query],
    n_results=2
)

res['documents'][0][0]

'define action execute transaction’s cycle. syntax event trn event_code endevent where: event_code code associate event. description event activate transaction end cycle; is: immediately commit. examples may want return call program enter transaction: event trn return endevent print card transaction transaction session, update last operation log file system, could program follow events: event trn printairlinecard.call(airlineid) updatelogairline.call(airlineid, &timeenter, &timeexit) endevent scope objects: transactions languages: .net, ruby, java, rpg, visual basic, visual foxpro, cobol see also aftercomplete event'

### Contexto

Para darle contexto a los modelos de lenguaje se deben elejir aquellos documentos que mas se relacionen con la pregunta del usuario con el fin de preparar previamente al modelo con información que le puede ser de utilidad para responder dicha pregunta.

Pero como un contexto voluminoso para un modelo puede no resultar benefisioso, la idea es separar los documentos cernanos en partes, y quedarse con las partes particulares de estos documentos que mas se parezcan a la pregunta inicial del usuario.

In [ ]:
def tokenize_sentences(res, raw=False):
    '''separa una documento de texto en oraciones'''
    sentences = []

    # si viene de chromadb
    if raw == False:
        for text_list in res['documents']:
            for text in text_list:
                # añiade a una lista todas las oraciones
                sentences.extend(sent_tokenize(text))
    # si es texto plano
    else:
        sentences.extend(sent_tokenize(res))

    return sentences

tokens_sentences = tokenize_sentences(res)

Para determinar cuales son las partes de los documentos mas cercanas al contexto de la pregunta, se llevan tanto las oraciones como la pregunta a un espacio vectorial que permitira determinar que tan cercanas son las oraciones de la pregunta, y con ello podemos quedarnos con un top de las mejores oraciones para utilizar como contexto.

In [ ]:
# se vectorizan las oraciones de contexto y la pregunta
sentence_embeddings = modelo_embedding.encode(tokens_sentences, convert_to_tensor=True)
query_embeddings = modelo_embedding.encode(user_query, convert_to_tensor=True)

# se obtiene la distancia de coseno entre la pregunta y cada oracion
puntuaciones_coseno = util.pytorch_cos_sim(query_embeddings, sentence_embeddings)[0]

# se almacena las tres oraciones mas cercanas
indices_ordenados = torch.argsort(puntuaciones_coseno, descending=True)
top_3_oraciones = [tokens_sentences[i] for i in indices_ordenados[:3]]

display(top_3_oraciones)
context = ' '.join(top_3_oraciones)

['syntax event trackcontext (parameters) event_code endevent where: event_code code associate event.',
 'syntax event trn event_code endevent where: event_code code associate event.',
 'define trackcontext event, allow retrieve context information take desire actions.']

### Modelo de Lenguaje

El modelo de lenguaje que se utilizara para simular conversiones entre un usuario con la documentación de Genexus sera Zephyr, un modelo de software libre disponible en HuggingFace para usar mediante RestAPI.

Este modelo se alimenta de un rol que el mismo debe cumplir en la conversación, el contexto para ubicarse para responder preguntas, y la pregunta del usuario como tál.

In [ ]:
# llamada al modelo mediante api
API_URL = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"
headers = {"Authorization": "HERE MUST BE YOUR HUGGINGFACE TOKEN!"}

# rol del agente
rol = 'You are a expert in Genexus 18 who always responds with the context provided using Genexus code syntax'

# envia la consulta al servidor y responde
def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

# estructura del input para el modelo
quest_to_model = f""" <|system|>
{rol}</s>
<|user|>
Context: {context}
Question: {user_query}
</s>
<|assistant|> """

# parametros del modelo
output = query({
	"inputs": quest_to_model,
	"parameters": {
		   "do_sample": False,
		   "max_new_tokens": 1000,
	}
})

In [ ]:
# ejemplo de respuesta
(output[0]['generated_text'][len(tino_quest): ].strip())

'The variable `&trnContext` is a built-in Genexus variable used in transaction events (`EVENT TRN`). It contains the context information of the current transaction, such as the user ID, company ID, and transaction ID. This variable can be accessed and manipulated in the transaction event code to perform specific actions based on the context of the transaction. For example, you can use `&trnContext` to log transaction details, validate user permissions, or perform auditing tasks. The syntax for accessing this variable is `&trnContext.propertyName`, where `propertyName` is the name of the property you want to access, such as `&trnContext.userId` or `&trnContext.companyId`.'

### Automatización de las consultas

La idea es que una unica llamada al modelo con una pregunta pueda resolver toda la lógica asociada en encontrar el contexto adecuado, y recibir la respuesta esperada en la salida.

In [ ]:
def get_contenxt(user, sentences, nearests=3):
    '''Obtiene de los documentos cernanos el top de oraciones mas cercanas al contexto'''
    # depurar la consulta
    user_query = clean_text(user)

    # vectorizacion del texto
    sentence_embeddings = modelo_embedding.encode(sentences, convert_to_tensor=True)
    query_embeddings = modelo_embedding.encode(user_query, convert_to_tensor=True)

    # obtener distancias
    puntuaciones_coseno = util.pytorch_cos_sim(query_embeddings, sentence_embeddings)[0]

    # obtener resultados mas cernados
    indices_ordenados = torch.argsort(puntuaciones_coseno, descending=True)
    top_3_oraciones = [sentences[i] for i in indices_ordenados[:nearests]]

    return ' '.join(top_3_oraciones)

def query_to_model(user, assistant, context=None, nearest=2):
    '''enviar consulta al modelo, determina el contexto y se le aniade
       otro si el usaurio quiere ingresarlo, y retorna la respuesta'''
    # buscar los documentos cercanos
    res = collection.query(
                           query_texts=[user],
                           n_results=nearest
                          )

    # extracion de los mejores fragmentos
    tokens_sentences = tokenize_sentences(res)
    space_context = get_contenxt(user, tokens_sentences, nearest)

    # aniadir contexto extra
    if context is not None:
        space_context += f' {context}'

    # estructua del input
    quest = f""" <|system|>
    {assistant}
    </s>
    <|user|>
    Context: {space_context}
    Question: {user}</s>
    <|assistant|> """

    # estructura de la consulta
    output = query({
        "inputs": quest,
        "parameters": {
            "do_sample": False,
            "max_new_tokens": 1000
        }
    })


    # obtiene la respuesta
    model_res = (output[0]['generated_text'][len(quest): ].strip())
    return model_res

question = 'Explain me about inout parameters in procedures'
print(query_to_model(user=question, assistant=rol))

Inout parameters in procedures allow passing values to a procedure and also allowing the procedure to modify those values. This is different from input parameters, where the procedure only receives the value, and output parameters, where the procedure returns a value.

Inout parameters are declared with the "inout" keyword in Genexus, and they can be used to pass complex data structures or large amounts of data between the procedure and the calling program. This can improve performance by avoiding the need to copy large amounts of data between the procedure and the calling program.

Here's an example of how to declare an inout parameter in Genexus:

```
procedure myProcedure(inout: myStruct)
begin
  // Modify the inout parameter here
end

// Call the procedure with an inout parameter
myStruct myStructVar;
myStructVar.field1 := 1;
myStructVar.field2 := "hello";
myProcedure(myStructVar);

// The inout parameter is now modified inside the procedure
// and the changes are reflected in the 

Se puede notar un respuesta bastante precisa al menos a nivel conceptual. Aunque al momento de generar código demuestra utilizar syntaxis que originalmente Genexus no utiliza, pero tampoco es tan ajeno la forma general del código a la respuesta esperada.

In [ ]:
# ejemplo de consulta mas especifica con contexto añiadido
question = '''Give a example of procedure that take a inout parameter that takes a number and return the number plus one defining the rules and the source of the procedure.'''
context = '''Inout parameters in procedures allow passing values to a procedure and also allowing the procedure to modify those values. This is different from input parameters, where the procedure only receives the value, and output parameters, where the procedure returns a value.'''
rol = '''You are an expert Genexus programer that responds wih shorts pieces of code.'''

print(query_to_model(user=question, assistant=rol, context=context, nearest=3))

Here's an example of a Genexus procedure that takes an inout parameter and returns the number plus one:

Procedure Name: IncrementNumber

Rules:

1. Declare the procedure with an inout parameter:

   DECLARE PROCEDURE IncrementNumber(INOUT num AS NUMBER)

2. Define the operator behavior:

   DEFINE OPERATE AS:

   INPUT PARAMETER: The parameter is being passed into the procedure, but its value can be modified inside the procedure.

   OUTPUT PARAMETER: The procedure returns a value, but the parameter's value is not modified inside the procedure.

   INPUT OUTPUT PARAMETER: The parameter is being passed into the procedure, and its value can be modified inside the procedure.

   In this case, we're using an inout parameter, so the behavior is:

   INPUT OUTPUT PARAMETER: The parameter is being passed into the procedure, and its value can be modified inside the procedure.

   Source:

   ```
   PROCEDURE IncrementNumber(INOUT num AS NUMBER)
   BEGIN
       num := num + 1;
   END
   ```

 

### Memoria de Corto Plazo

La idea es que el modelo pueda mantener la coherencia de la conversión respecto a las consultas y respuestas previas. Para ello se plantea una estructura de datos para el contexto como una lista de strings que se modificando a lo largo de la conversación, pero manteniendo el contexto reciente en memoria.

Para ello se define una cola (Queue) donde se almacena el contexto de los documentos mas el contexto de la conversación.

In [ ]:
global_context = []

Se hace una modifación sobre esta función para retornar una lista de oraciones cernadas en lugar de un string concatenado.

In [ ]:
def get_contenxt(user, sentences, nearests=3):
    '''Obtiene de los documentos cernanos el top de oraciones mas cercanas al contexto'''
    # depurar la consulta
    user_query = clean_text(user)

    # vectorizacion del texto
    sentence_embeddings = modelo_embedding.encode(sentences, convert_to_tensor=True)
    query_embeddings = modelo_embedding.encode(user_query, convert_to_tensor=True)

    # obtener distancias
    puntuaciones_coseno = util.pytorch_cos_sim(query_embeddings, sentence_embeddings)[0]

    # obtener resultados mas cernados
    indices_ordenados = torch.argsort(puntuaciones_coseno, descending=True)
    top_3_oraciones = [sentences[i] for i in indices_ordenados[:nearests]]

    return top_3_oraciones

Una memoria de 8 unidades implica que contemplará las 3 oraciones de la documentación mas cernanas a la pregunta previa, las tres oraciones mas cercanas a la pregunta actual en la documentación, y las tres mas cercanas al texto de la respuesta previa.

In [ ]:
def query_to_model(user, assistant, context=None, nearest=3, memory=8):
    '''enviar consulta al modelo, determina el contexto y se le añade
       contexto de la conversacion, y retorna la respuesta'''
    global global_context

    # buscar los documentos cercanos
    res = collection.query(
                           query_texts=[user],
                           n_results=nearest
                          )

    # extracion de los mejores fragmentos de la documentación
    tokens_sentences = tokenize_sentences(res)
    space_context = get_contenxt(user, tokens_sentences)

    # añiade al contexto global
    for sentence in space_context:
        global_context.append(sentence)

    if context is not None:
        # depurar el contexto de conversacion
        clean_chat_context = clean_text(context)

        # extracion de los mejores fragmentos de la respuesta previa
        tokens_chat = tokenize_sentences(clean_chat_context, raw=True)
        chat_context_sentences = get_contenxt(user, tokens_chat)

        # añiade al contexto global
        for sentence_chat in chat_context_sentences:
            global_context.append(sentence_chat)

    # remueve los contextos mas viejos
    while len(global_context) > memory:
        global_context.pop(0)

    context_model = ' '.join(global_context)

    # estructua del input
    quest = f""" <|system|>
    {assistant}
    </s>
    <|user|>
    Context: {context_model}
    Question: {user}</s>
    <|assistant|> """

    # estructura de la consulta
    output = query({
        "inputs": quest,
        "parameters": {
            "do_sample": False,
            "max_new_tokens": 1000
        }
    })


    # obtiene la respuesta
    model_res = (output[0]['generated_text'][len(quest): ].strip())
    return model_res

question = 'Explain me about inout parameters in procedures'
model_answer = query_to_model(user=question, assistant=rol)
print(model_answer)

Inout parameters in procedures allow passing values to the procedure and also allowing the procedure to modify those values. This is different from input parameters, where the procedure can only receive values, and output parameters, where the procedure can only return values.

Inout parameters are declared with the "inout" keyword in the parameter list of the procedure. When a variable is passed as an inout parameter, it is initially passed by value, but any changes made to the variable within the procedure are reflected in the original variable outside the procedure.

This feature is useful when you want to modify the original variable based on some calculation or operation performed within the procedure. It saves the need to pass the modified variable back as an output parameter, which can be more efficient and less error-prone.

Here's an example:

```gen
procedure sum(inout num1, inout num2)
    var sum as integer
    sum = num1 + num2
    num1 = num1 - num2
    num2 = sum
end

//

In [ ]:
global_context

['example follow rule declare procedure: parm(out:&var1, in:&var2, &var3, inout:&var4); so, note that: &var1: parameter &var2: parameter &var3: in-out parameter &var4: in-out parameter advantage define operators declare explicitly parameter behavior, obtain follow advantages: 1. better specification semantics interface.',
 'parameter declare parm rule define operate as: input parameter (specifying operator) output parameter (specifying operator) input output parameter (specifying inout operator) operator specified, depend following: object call call, parameter behavior be: inout.',
 'parameters receive null value.']

Notar como en la primera pregunta solo utiliza de contexto los documentos.

In [ ]:
question2 = 'how can i do the sum if parameters are only input parameters?'
second_answer = query_to_model(user=question2, assistant=rol, context=model_answer)

In [ ]:
global_context

['parameter declare parm rule define operate as: input parameter (specifying operator) output parameter (specifying operator) input output parameter (specifying inout operator) operator specified, depend following: object call call, parameter behavior be: inout.',
 'parameters receive null value.',
 "note count formulas, first parameter can't expression.",
 'user variables use inline formulas.',
 'sum count formulas, result evaluate expression must numeric value.',
 'inout parameters declare "inout" keyword parameter list procedure.',
 'different input parameters, procedure receive values, output parameters, procedure return values.',
 "here's example: ```gen procedure sum(inout num1, inout num2) var sum integer sum = num1 + num2 num1 = num1 - num2 num2 = sum end // call procedure var a, b integer = 10 b = 20 sum(a, b) // call procedure, b new value // = 50 (10 - 20) // b = 30 (10 + 20) ``` example, `sum` procedure take two inout parameters `num1` `num2`, perform calculations, modify o

Pero para la segunda pregunta el contexto ahora abarca tambien la nueva documentación y la respuesta previa.

In [ ]:
print(second_answer)

If the parameters are only input parameters, you cannot modify their values within the procedure. Instead, you can create a new output parameter to hold the sum and return it at the end of the procedure. Here's an example:

```gen
procedure sum(input num1, input num2)
var sum integer
sum = num1 + num2
return sum
end

// Call procedure and store result in output parameter
var result integer
sum(10, 20, result)
```

In this example, the `sum` procedure takes two input parameters `num1` and `num2`, calculates their sum, and returns it as an output parameter `sum`. When calling the procedure, you pass the input parameters and a new output parameter `result` to store the sum.


### Traducción

Debido a que la documentación de Genexus está en inglés y no se puede obtener directamente en español, se opta por añadir un paso más a la hora de interactuar, que es la traducción automática del texto tanto ingresado como retornado, debido a en la interfaz el usuario debe poder conversar en español, pero el modelo de fondo trabaja en inglés.

In [ ]:
rol_en = GoogleTranslator(source='es', target='en').translate("Eres un programador experto Genexus que siempre responde utilizando el contexto proporcionado.")
print(rol_en)

rol_es = GoogleTranslator(source='en', target='es').translate("You are an Genexus expert programer that always responds using the provided context.")
print(rol_es)

You are an expert Genexus programmer who always responds using the context provided.
Eres un programador experto Genexus que siempre responde utilizando el contexto proporcionado.


Un problema que se presenta es que el código no debe ser traducido ya que usa palabras reservadas del lenguaje. Para ello se crea la siguiente función para evitar la traducción en bloques de texto.

In [ ]:
# Expresión regular para identificar bloques de código
code_block_pattern = re.compile(r'`{3}.*?`{3}', re.DOTALL)

def translate_except_code(text, source, target):
    '''Traducir al español la respuesta, excluyendo bloques de código'''
    # Encuentra todos los bloques de código en el texto
    code_blocks = code_block_pattern.findall(text)

    # Reemplaza los bloques de código con marcadores temporales
    for i, block in enumerate(code_blocks):
        text = text.replace(block, f'__CODE_BLOCK_{i}__')

    # Traduce el texto sin los bloques de código
    translated_text = GoogleTranslator(source=source, target=target).translate(text)

    # Restaura los bloques de código en la traducción
    for i, block in enumerate(code_blocks):
        translated_text = translated_text.replace(f'__CODE_BLOCK_{i}__', block)

    return translated_text

example_with_code = '''
3. In the details view, click the "Formula" tab to open the formula editor.
4. Click the "New Formula" button to create a new formula.
5. In the formula editor, enter the following formula:
```
Sum(Price, Where(Level = Level - 1))
```
6. Save the formula and close the formula editor.
7. Open the entity diagram where you defined the global formula.'''

model_answer_es = translate_except_code(example_with_code, 'en', 'es')
print(model_answer_es)

3. En la vista de detalles, haga clic en la pestaña "Fórmula" para abrir el editor de fórmulas.
4. Haga clic en el botón "Nueva fórmula" para crear una nueva fórmula.
5. En el editor de fórmulas, ingrese la siguiente fórmula:
```
Sum(Price, Where(Level = Level - 1))
```
6. Guarde la fórmula y cierre el editor de fórmulas.
7. Abra el diagrama de entidad donde definió la fórmula global.


### Conversación

Finalmente se logra un modelo de conversación que permite ingresar preguntas en español sobre algún tema de Genexus, y el modelo de lenguaje tomara información relacionada de las fuentes de datos para tratar de reponder de forma más precisa en español mantiendo la coherencia para los bloques de código.

In [ ]:
# inicializa el contexto y rol
global_context = []
rol = 'You are an Genexus expert programer that always responds using the provided context with brief responses and short pieces of code.'
first_question = True

# iteracion de la conversacion
while True:
    user_input = input("\u0001 [User]: ")
    user_input_en = GoogleTranslator(source='es', target='en').translate(user_input)

    print('\n')

    # keyword de escape
    if user_input.lower() == 'exit':
        print("Exiting the conversation.")
        break

    # obtiene respuesta de acuerdo de acuerdo a si usa o no contexto previo
    if first_question:
        model_answer = query_to_model(user_input, rol, memory=6)
        first_question = False
    else:
        model_answer = query_to_model(user_input, rol, context=model_answer, memory=6)

    # traducir al español la respuesta
    model_answer_es = translate_except_code(model_answer, 'en', 'es')

    print("\u0001 [Model]:", model_answer_es, '\n')

 [User]: Enseniame a hacer un procedimiento que tome dos numeros y retorno su suma


 [Model]: Aquí hay un procedimiento Genexus que toma dos números como entrada y devuelve su suma:

1. Cree un nuevo procedimiento en Genexus y asígnele el nombre "AddNumbers".
2. Agregue dos parámetros de entrada de tipo numérico, denominados "num1" y "num2".
3. Agregue un parámetro de salida de tipo numérico, denominado "suma".
4. En el cuerpo del procedimiento, agregue el siguiente código:

```
sum = num1 + num2
```

5. Guarde y pruebe el procedimiento.

Aquí está la sintaxis para llamar al procedimiento:

```
CALL PROCEDURE("AddNumbers") PARAM("num1", <num1 value>) PARAM("num2", <num2 value>) RESULT("sum")
```

Reemplace `<valor num1>` y `<valor num2>` con los valores reales que desea agregar. El resultado del procedimiento se almacenará en la variable `suma`. 

 [User]: Esta bien. Y ahora si quisiera que divida los numeros de la entrada en lugar de sumar? debes contemplar el caso de que el divis